# Jupyter Notebook

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

In [7]:
twenty_twenty_two_overalladp = pd.read_csv('data/FantasyPros_2022_Overall_ADP_Rankings.csv')
twenty_twenty_two_overalladp

,Rank,Player,Team,Bye,POS,ESPN,RTSports,MFL,Fantrax,FFC,Sleeper,AVG
0,1.0,Jonathan Taylor,IND,14.0,RB1,1.0,1.0,NaN,NaN,NaN,1.0,1.0
1,2.0,Christian McCaffrey,CAR,13.0,RB2,2.0,2.0,NaN,NaN,NaN,2.0,2.0
2,3.0,Austin Ekeler,LAC,8.0,RB3,4.0,3.0,NaN,NaN,NaN,4.0,3.8
3,4.0,Cooper Kupp,LAR,7.0,WR1,3.0,5.0,NaN,NaN,NaN,5.0,4.0
4,5.0,Derrick Henry,TEN,6.0,RB4,5.0,8.0,NaN,NaN,NaN,3.0,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...
350,351.0,Malik Willis,TEN,6.0,QB39,NaN,NaN,NaN,NaN,NaN,NaN,297.0
351,352.0,Joey Slye,WAS,14.0,K29,NaN,NaN,NaN,NaN,NaN,NaN,299.0
352,353.0,O.J. Howard,HOU,6.0,TE44,NaN,NaN,NaN,NaN,NaN,NaN,300.0
353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
twenty_eighteen_stats_wrstats = pd.read_csv('data/fantasy_pros_2018_wr_statistics.csv')
twenty_eighteen_stats_wrstats

,Rank,Player,G,REC,YDS,Y/R,YBC,YBC/R,AIR,AIR/R,...,% TM,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,LNG
0,1.0,Tyreek Hill (MIA),16.0,87.0,"1,479",17.0,952,10.9,952,10.9,...,27.1%,94.0,6.0,9.0,50.0,27.0,16.0,8.0,4.0,75.0
1,2.0,Antonio Brown (FA),15.0,104.0,"1,297",12.5,812,7.8,812,7.8,...,24.9%,107.0,3.0,12.0,48.0,17.0,8.0,6.0,2.0,78.0
2,3.0,Davante Adams (LV),15.0,111.0,"1,386",12.5,911,8.2,911,8.2,...,27.5%,116.0,5.0,15.0,53.0,18.0,8.0,5.0,2.0,57.0
3,4.0,DeAndre Hopkins (ARI),16.0,115.0,"1,572",13.7,"1,185",10.3,"1,185",10.3,...,33.0%,121.0,2.0,10.0,66.0,23.0,9.0,4.0,0.0,49.0
4,5.0,Julio Jones (TB),16.0,113.0,"1,677",14.8,"1,229",10.9,"1,229",10.9,...,28.1%,123.0,9.0,13.0,78.0,24.0,10.0,2.0,1.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,182.0,Josh Malone (PIT),9.0,1.0,12,12.0,12,12.0,12,12.0,...,0.9%,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,12.0
182,183.0,Victor Bolden Jr. (FA),6.0,1.0,10,10.0,9,9.0,9,9.0,...,0.8%,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,10.0
183,184.0,Jeremy Kerley (FA),1.0,2.0,7,3.5,-8,-4.0,-8,-4.0,...,0.7%,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
overall_adp_2019 = pd.read_csv('data/fantasy_pros_2018_wr_statistics.csv')
wr_rankings_2019 = pd.read_csv('data/')